In [38]:
import numpy as np
import pandas as pd
import json
import os

In [52]:
# places one of each 'eventType' into a json file
# this is just for testing purposes... essentially useless LOL
eventType = set()
dataDict = []

with open('Data/games/ESPORTSTMNT03_3195276.json', 'r', encoding='utf-8') as file:
    game_data = json.load(file)

# if isinstance(game_data, list):
#     game_data.reverse()

for entry in game_data:
    eventType.add(entry['eventType'])

for entry in game_data:
    if entry['eventType'] in eventType:
        eventType.remove(entry['eventType'])
        dataDict.append(entry)
    if len(eventType) == 0:
        break

with open('Data/help.json', 'w') as json_file:
    json.dump(dataDict, json_file)


In [75]:
# modify teams.json so i get go from team id to team name

teams = dict()

with open('Data/teams.json', 'r', encoding='utf-8') as file:
    mapping_data = json.load(file)

for entry in mapping_data:
    teams[entry['team_id']] = entry['name']

with open('Data/teams_but_better.json', 'w') as json_file:
    json.dump(teams, json_file)

In [39]:
# modify mapping_data so it suites team mapping better :D
game_mapping_data = dict()

with open('Data/mapping_data.json', 'r', encoding='utf-8') as file:
    mapping_data = json.load(file)

for entry in mapping_data:
    game_mapping_data[entry['platformGameId']] = entry['teamMapping']

with open('Data/game_mapping_data.json', 'w') as json_file:
    json.dump(game_mapping_data, json_file)

In [57]:
team_data_dict = dict()

with open('Data/game_mapping_data.json', 'r') as json_file:
    team_mapping_data = json.load(json_file)

# Iterate through all files in the directory
for filename in os.listdir('games'):
    # Create the full path to the file
    file_path = os.path.join('games', filename)

    with open(file_path, 'r', encoding='utf-8') as json_file:
        game_data = json.load(json_file)


    # we want: game_length, total_kills, total_assists, total_deaths, total_damage, total_gold, total_level, win or loss
    game_id = game_data["game_end"]['platformGameId']
    team_mapping = team_mapping_data[game_id]
    winning_team = str(game_data["game_end"]["winningTeam"])
    team_stats = {str(team['teamID']): team for team in game_data["end_stats"]}

    for key in team_stats:
        team_stats[key]["won"] = key == winning_team
        team_stats[key]["gameTime"] = game_data["game_end"]["gameTime"]

    for key, value in team_stats.items():
        # print(type(key))
        team_id = team_mapping[key]
        if team_id in team_data_dict:
            team_data_dict[team_id].append(value)
        else:
            team_data_dict[team_id] = [value]

    print(team_stats)


with open('Data/msi_2023_team_stats.json', 'w') as json_file:
    json.dump(team_data_dict, json_file)



{'100': {'inhibKills': 0, 'towerKills': 1, 'teamID': 100, 'baronKills': 0, 'assists': 22, 'dragonKills': 0, 'championsKills': 8, 'totalGold': 45046, 'deaths': 20, 'totalDamage': 549107.033203125, 'totalLevel': 65, 'won': False, 'gameTime': 1616432}, '200': {'inhibKills': 2, 'towerKills': 9, 'teamID': 200, 'baronKills': 1, 'assists': 44, 'dragonKills': 4, 'championsKills': 20, 'totalGold': 56865, 'deaths': 8, 'totalDamage': 699357.548828125, 'totalLevel': 73, 'won': True, 'gameTime': 1616432}}
{'100': {'inhibKills': 0, 'towerKills': 4, 'teamID': 100, 'baronKills': 0, 'dragonKills': 3, 'assists': 29, 'championsKills': 11, 'totalGold': 67739, 'deaths': 18, 'totalDamage': 1023992.51953125, 'totalLevel': 82, 'won': False, 'gameTime': 2498912}, '200': {'inhibKills': 1, 'towerKills': 7, 'teamID': 200, 'baronKills': 2, 'dragonKills': 4, 'assists': 46, 'championsKills': 18, 'totalGold': 76307, 'deaths': 11, 'totalDamage': 1286963.44140625, 'totalLevel': 87, 'won': True, 'gameTime': 2498912}}
{'

In [53]:
count = 0
for filename in os.listdir('games/'):
    count += 1

print(count)

76


In [63]:
# average all the stats for each team

team_average_data = dict()

with open("Data/msi_2023_team_stats.json", 'r', encoding='utf-8') as json_file:
    team_stats = json.load(json_file)

for team, stats_list in team_stats.items():
    average_stats = {
      "inhibKills": 0,
      "towerKills": 0,
      "baronKills": 0,
      "assists": 0,
      "dragonKills": 0,
      "championsKills": 0,
      "totalGold": 0,
      "deaths": 0,
      "totalDamage": 0,
      "totalLevel": 0,
      "gameTime": 0
    }

    for key in average_stats.keys():
        average_stats[key] = sum(stats[key] for stats in stats_list)
        average_stats[key] /= len(stats_list)

    average_stats["wins"] = sum(stats["won"] for stats in stats_list)
    average_stats["losses"] = len(stats_list) - average_stats["wins"]

    team_average_data[team] = average_stats

with open('Data/msi_2023_average_team_stats.json', 'w') as json_file:
    json.dump(team_average_data, json_file)


In [76]:
# using this formula for all the teams
# game_grade = 0.336 — (1.437 x Deaths per min) + (0.000117 x gold per min) + (0.443 x K_A per min) + (0.264 x Level per min) + (0.000013 x TD per min)
from pprint import pprint
def calculate_grade(data):
    game_time = data["gameTime"] / 60000
    return 0.336 - (1.437 * (data["deaths"]/game_time)) + (0.000117 * (data["totalGold"]/game_time)) + (0.443 * ((data["assists"] + data["championsKills"])/game_time)) + (0.264 *  (data["totalLevel"]/game_time)) + (0.000013 * (data["totalDamage"]/game_time))

team_grade_dict = dict()

with open("Data/msi_2023_average_team_stats.json", 'r', encoding='utf-8') as json_file:
    team_average_stats = json.load(json_file)

with open("Data/teams_but_better.json", 'r', encoding='utf-8') as json_file:
    team_name_mapping = json.load(json_file)


for team, stats in team_average_stats.items():
    team_grade_dict[team_name_mapping[team]] = calculate_grade(stats)


sorted_dict = sorted(team_grade_dict.items(), key=lambda item: item[1], reverse=True)

pprint(sorted_dict)


[('Beijing JDG Intel Esports Club', 1.9653594654412818),
 ('Bilibili Gaming Pingan Bank', 1.6676915532823182),
 ('G2 Esports', 1.5754023362648737),
 ('Golden Guardians', 1.5721911369896167),
 ('Gen.G', 1.5590799162213564),
 ('T1', 1.4949498231538805),
 ('PSG Talon', 1.4322978679697056),
 ('Cloud9', 1.2741438416522382),
 ('GAM Esports', 1.2520488864903843),
 ('LOUD', 1.1627047578845437),
 ('DetonatioN FocusMe', 1.093296009574352),
 ('Movistar R7', 0.9679703133440348),
 ('MAD Lions', 0.803984159899297)]
